In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import platform as pl
import plotly.express as px
import plotly.graph_objects as go
import random
from datetime import datetime


In [3]:
# file = "logs_abdi_2.csv"
file = "dados_sim.csv"
escavadeira = 24
caminhao = 25

sep = '\\' if pl.uname().system == 'Windows' else '/'
df = pd.read_csv(f"..{sep}dados_csv{sep}{file}", encoding='iso-8859-1')
df.loc[:,'date_created'] = pd.to_datetime(df['date_created'])

litragem_cam = 210
litragem_esc = 345


### Dados embasados nas especificações técnicas dos veículos. São elas:
Dados do Caminhão:
 - Modelo: Mercedes 2423k (mb 2423)
 - Bateria (qtde x Ah/V): 2 x 135/12
 - Volume Caçamba: 12 m³
 - Velocidade Máxima: 82 Km/h
 - Capacidade Subida (em mov.): 55% * 23t
 - Capacidade Subida (parado): 39% * 23t
 - Carga Máxima útil: 16,050 t
 - Capacidade Combustível: 210 L
 - Consumo Combustível: 1,5 Km/L a 3 Km/L


Dados da Escavadeira:
 - Modelo: Cat320D
  - Bateria
  - Capacidade Combustível: 345 L
  - Capascidade da Caçamba: 732 Kg (1,19 m³)
  - Autonomia: 9 L/h a 12 L/h
  - Tempo de Carga de um Caminhao: 4min a 8min


# Fator de Eficiência
O Fator de Eficiência leva em consideração os tempos efetivamente trabalhados pela escavadeira medido em uma hora, definindo sua relação em um indicador (E) adimensional. A periodicidade de aferição será ao final de cada turno de trabalho.
- E= Tempo efetivamente trabalhados (min) / 60 (min)   

In [4]:
# df_data = df.loc[(df['imei'] == escavadeira)]
# df_data = df.loc[df['date_created'] < '2021-08-07 00:00:00']
df_data = df.copy()
df_data.loc[:,'date_created'] = pd.to_datetime(df_data['date_created'])
df_data['last_date'] = df_data['date_created'].shift(1)
df_data['interval_'] = (df_data['date_created'] - df_data['last_date']) / np.timedelta64(1, 'm')
df_data = df_data.set_index('date_created')
df_data = df_data.loc['2021-08-06':]

df_total = df_data.copy()
df_data = df_data.loc[df_data['weight'] > 0]
# Agrupa por hora
df_data = df_data.groupby(pd.Grouper(freq='3600s')).sum()
df_total = df_total.groupby(pd.Grouper(freq='3600s')).max()

df_total['eficiencia'] = df_data['interval_'].apply(lambda x: round(x/60, 3)) # Eficiencia: intervalo/hora em (min)
df_total['eficiencia'].fillna(0, inplace=True)

fig = px.bar(df_total, y="eficiencia", title="Fator de Eficiencia", text=df_total['eficiencia'])
fig.update_traces(texttemplate='%{text:.2}', textposition='outside')
fig.show()

## Rendimento

O Rendimento (R) mostrará a quantidade de material movimentado pelo equipamento em uma média diária.
    
- R = Somatório(pesos)/Dia (Kg/h)


In [5]:
df_rend = df_total.copy()
# Agrupa por dia
df_rend = df_rend.groupby(pd.Grouper(freq='86400s')).sum()
fig = px.bar(df_rend, y="weight", title="Gráfico de Rendimento (Kg/dia)", text=df_rend['weight'])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.show()

## Aferição do tempo de carga
Aferição do Tempo de Carga (Tc) de uma escavadeira para a realização de uma operação no canteiro de obra.
- Tc = Tempo ciclo de um equipamento (h)


In [6]:
import plotly.graph_objects as go

df_temp = df.copy()
df_temp = df_temp.loc[df_temp['date_created'] < '2021-08-07 00:00:00']
df_temp.rename(columns = {'interval':'load_time'}, inplace = True)
# df_temp = df_temp.loc[df_temp['weight'] > 0, ['weight', 'load_time']]
df_temp['weight'] = df_temp['weight'].apply(lambda x: round(x/1000, 3))

last_status = df_temp.iloc[0,1]
weight_acc = 0 #Peso acumulado
interval_sum = 0 #Intervalo do ciclo
init_count = False

def LoadingCycle(df):
    global last_status, weight_acc, interval_sum, init_count
    interval_per_cycle = None
    weight_sum = None
    cur_status = df['status']
    
    # Inicia Ciclo de Carregamento
    if (last_status == 'Parado' or last_status == 'Movimento' or last_status == 'Descarregando') and (cur_status == 'Carregando'):
        init_count = True
    # Finaliza Ciclo de Carregamento
    if (cur_status == 'Parado' or cur_status == 'Movimento' or cur_status == 'Descarregando') and (last_status == 'Carregando'):
        init_count = False
        interval_per_cycle = interval_sum
        weight_sum = weight_acc
        weight_acc = 0
        interval_sum = 0
    
    if init_count == True:
        weight_acc += df['weight']
        interval_sum += df['load_time']
    last_status = cur_status
    return [interval_per_cycle, weight_sum]


df_temp['interval/cycle'] = df_temp.apply(lambda x: LoadingCycle(x)[0], axis=1)
df_temp['weight_sum'] = df_temp.apply(lambda x: LoadingCycle(x)[1], axis=1)
df_temp = df_temp.loc[df_temp['interval/cycle'] > 0]
# display(df_temp)

fig = go.Figure()
fig.add_trace(go.Bar(x=df_temp['date_created'],
                y=df_temp['interval/cycle'],
                name='Tempo em Minutos',
                marker_color='rgb(55, 83, 109)',
                text=df_temp['interval/cycle']
                ))
fig.add_trace(go.Bar(x=df_temp['date_created'],
                y=df_temp['weight_sum'],
                name='Carga em Toneladas',
                marker_color='rgb(26, 118, 255)',
                text=df_temp['weight_sum']
                ))

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(
    title='Aferição do tempo de Carga',
    xaxis_tickfont_size=14,
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)

fig.show()

## Consumo de Combustível da Escavadeira
O Consumo de Combustível é o cálculo do volume consumido em relação a um período de tempo. 

Forma de Cálculo: Média de consumo de combustível no dia
- Cc = V combustível / T operação (L/dia ou L/h)

In [19]:
df_comb = df_total.copy()
df_max = df_comb.groupby(pd.Grouper(freq='86400s')).max().fillna(0)
df_min = df_comb.groupby(pd.Grouper(freq='86400s')).min().fillna(0)
df_comb = df_comb.groupby(pd.Grouper(freq='86400s')).max().fillna(0)
diff_fuel = df_max['fuel_esc'] - df_min['fuel_esc']
df_comb['diff_fuel'] = diff_fuel
df_comb['consumo'] = (diff_fuel*litragem_esc)/100
# print(df_comb.loc[:, ['diff_fuel', 'consumo']])

fig = go.Figure()
fig.add_trace(go.Bar(x=df_comb.index,
                y=df_comb['diff_fuel'],
                name='Consumo do Combustivel em (%)',
                marker_color='rgb(55, 83, 109)',
                text = df_comb['diff_fuel']
                ))
fig.add_trace(go.Bar(x=df_comb.index,
                y=df_comb['consumo'],
                name='Consumo em Litros (L)',
                marker_color='rgb(26, 118, 255)',
                text = df_comb['consumo']
                ))

fig.update_traces(texttemplate='%{text:.2}', textposition='outside')
fig.update_layout(
    title='Consumo de Combustível da Escavadeira',
    xaxis_tickfont_size=14,
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

## Consumo de Combustível do Caminhão
O Consumo de Combustível é o cálculo do volume consumido em relação a um período de tempo. 

Forma de Cálculo: Média de consumo de combustível no dia
- Cc = V combustível / T operação (L/dia ou L/h)


In [8]:
# df_data = df.loc[(df['imei'] == caminhao)]
df_data = df.copy()
df_data.loc[:,'date_created'] = pd.to_datetime(df_data['date_created'])
df_data = df_data.set_index('date_created')
df_data = df_data.loc['2021-08-06':]

df_comb = df_data.copy()
litragem = 300
df_max = df_comb.groupby(pd.Grouper(freq='86400s')).max().fillna(0)
df_min = df_comb.groupby(pd.Grouper(freq='86400s')).min().fillna(0)
df_comb = df_comb.groupby(pd.Grouper(freq='86400s')).max().fillna(0)
diff_fuel = df_max['fuel'] - df_min['fuel']
df_comb['diff_fuel'] = diff_fuel
df_comb['consumo'] = (diff_fuel*litragem)/100
# print(df_comb.loc[:, ['diff_fuel', 'consumo']])

fig = go.Figure()
fig.add_trace(go.Bar(x=df_comb.index,
                y=df_comb['diff_fuel'],
                name='Consumo de Combustivel (%)',
                marker_color='rgb(55, 83, 109)',
                text = df_comb['diff_fuel']
                ))
fig.add_trace(go.Bar(x=df_comb.index,
                y=df_comb['consumo'],
                name='Consumo em Litros (L)',
                marker_color='rgb(26, 118, 255)',
                text = df_comb['consumo']
                ))

fig.update_layout(
    title='Consumo de Combustível do Caminhão',
    xaxis_tickfont_size=14,
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

## Peso Transportado por Combustível Consumido

A relação entre os dois dados permitirá a verificação da eficácia de maquinários nas realizações das operações elencadas.

* Forma de Cálculo: Peso Transportado por um equipamento em relação à quantidade de combustível consumido em determinado trecho
* Unidade: Kg/L
* PtCC = Massa Transportada (Kg) / Volume Consumido (L)

In [21]:
# df_transp = df.copy()
df_transp = df.loc[df['date_created'] < '2021-08-07 00:00:00']
# display(df_transp)
last_status = df_transp.iloc[0,1]
weight_acc = 0 #Peso acumulado

def WeightPerFuel(df):
    global last_status, weight_acc
    weight_per_fuel = None
    cur_status = df['status']
    if (last_status == 'Parado' or last_status == 'Movimento' or last_status == 'Descarregando') and (cur_status == 'Carregando'):
        weight_acc += df['weight']
    if (cur_status == 'Movimento') and (last_status == 'Carregando'):
        weight_per_fuel = round(weight_acc/df['fuel_comsumed'],2)
        weight_acc = 0
    last_status = cur_status
    return weight_per_fuel

df_transp['W/F'] = df_transp.apply(lambda x: WeightPerFuel(x), axis=1)
df_transp = df_transp.dropna(axis=0)
# display(df_transp) 
fig = px.line(df_transp, x="date_created", y="W/F", title="Peso Transportado por Combustível Comsumido (Kg/L)")
fig.show()


## Trajeto Ciclo de Carga

In [10]:
# Plota Gráfico
def plotDatas(fig, datas, is_line=True, label=''):
        if fig is None:
            fig = go.Figure(go.Scattermapbox(
            mode = "markers+lines" if is_line == True else "markers",
            lon = datas[:,1],
            lat = datas[:,0],
            text = label if label != '' else 'veic1',
            marker = {'size': 10,
                    'color': 'blue'}))
        else:
            fig.add_trace(go.Scattermapbox(
            mode = "markers+lines" if is_line == True else "markers",
            lon = datas[:,1],
            lat = datas[:,0],
            text = label if label != '' else 'veic2',
            marker = {'size': 10,
                    'color': 'red'}))    

        fig.update_layout(
            margin ={'l':0,'t':0,'b':0,'r':0},
            #'center': {'lon': -38.71901, 'lat':  -3.94467},
            mapbox = {
                'center': {'lon': datas[:,1].mean(), 'lat':  datas[:,0].mean()},
                'style': "stamen-terrain",
                'center': {'lon': datas[:,1].mean(), 'lat':  datas[:,0].mean()},
                'zoom': 16})
        return fig

# Geração de Dados
### Trajeto do Caminhão dos dias 05/08 a 18/08

In [11]:
# import gmaps
from datetime import datetime
df_interval = df.loc[(df['imei'] == caminhao)]
df_gps = df.loc[:, ['latitude','longitude']]
gps_tuples = [tuple(x) for x in df_gps.to_numpy()]
start = gps_tuples[0]
end = gps_tuples[1]
# api_key_google = open('../api-google.txt').read()
# gmaps.configure(api_key=api_key_google)
# fig = gmaps.figure()
# layer = gmaps.directions.Directions(start, end,waypoints=gps_tuples)
# markers = gmaps.marker_layer(gps_tuples)
# layer = gmaps.symbol_layer(df_gps, fill_color = 'red', stroke_color='red', scale=2)
# fig.add_layer(layer)
# fig
def plotDataTimestamp(df):
    subset = df.loc[:, ['date_created','velocity','fuel']]
    text_df = [tuple(x) for x in subset.to_numpy()]
    fig = go.Figure(go.Scattermapbox(
        lat = df['latitude'].to_numpy(),
        lon = df['longitude'].to_numpy(),
        mode = 'markers',
        marker = go.scattermapbox.Marker( size = 9),
        # text = df['date_created'].astype('string').to_numpy()
        text = text_df
    ))

    mapbox_access_token = open('../mapbox_token.txt').read()

    fig.update_layout(
        margin ={'l':0,'t':0,'b':0,'r':0},
        #'center': {'lon': -38.71901, 'lat':  -3.94467},
        mapbox = {
            'center': {'lon': df['longitude'].to_numpy()[0], 'lat':  df['latitude'].to_numpy()[0]},
            'style': "stamen-terrain",
            'center': {'lon': df['longitude'].to_numpy()[0], 'lat':  df['latitude'].to_numpy()[0]},
            'zoom': 16})

    fig.show()


plotDataTimestamp(df_interval)



KeyError: 'imei'

### Trajeto de Abastecimento

In [ ]:
from geopy.distance import great_circle

df_abast = df.copy()
df_abast = df_abast.loc[(df['imei'] == caminhao) & 
                        (df['date_created'] >= '2021-08-07 12:29:18') &
                        (df['date_created'] <= '2021-08-07 12:41:47')]


timestamp = df_abast['date_created'].to_numpy()
interval = float((timestamp[-1] - timestamp[0]) / (10**9))
location = df_abast.loc[:,['latitude','longitude']].to_numpy()
distance = great_circle(location[0], location[-1]).m
vel_media = (distance*3600)/(1000*interval)
print(f'Total Distance: {round(distance, 2)} m, Interval: {round((interval / 60), 2)} min, Vel Media: {round(vel_media, 2)} Km/h')
plotDataTimestamp(df_abast)

Total Distance: 3920.73 m, Interval: 12.48 min, Vel Media: 18.84 Km/h


### Trajeto ao Ponto de Apoio 


In [ ]:
from geopy.distance import great_circle
from IPython.display import display, HTML
import random

litragem_cam = 210
df_apoio = df.copy()
df_apoio = df_apoio.loc[(df['imei'] == caminhao) & 
                        (df['date_created'] >= '2021-08-07 08:50:23') &
                        (df['date_created'] <= '2021-08-07 08:55:50')]


timestamp = df_apoio['date_created'].to_numpy()
interval = float((timestamp[-1] - timestamp[0]) / (10**9))
location = df_apoio.loc[:,['latitude','longitude']].to_numpy()
distance = great_circle(location[0], location[-1]).m
vel_media = (distance*3600)/(1000*interval)
autonomia = (distance/1000)/(0.71*litragem_cam/100)
print(f'Total Distance: {round(distance, 2)} m, Interval: {round((interval / 60), 2)} min, \
        Vel Media: {round(vel_media, 2)} Km/h, Autonomia: {round(autonomia, 2)} Km/L, max: {int(df["fuel"].max())}, min: {int(df["fuel"].min())}')

last_lat = df_apoio['latitude'].to_numpy()[0]
last_lon = df_apoio['longitude'].to_numpy()[0]

def calcDistance(df):
        global last_lat, last_lon
        dist = great_circle([df['latitude'],df['longitude']], [last_lat, last_lon]).m
        last_lat = df['latitude']
        last_lon = df['longitude']
        return dist

df_apoio['distance'] = df_apoio.apply(lambda x: calcDistance(x), axis=1)

nivel_comb = df_apoio['fuel'].to_numpy()[0]*litragem_cam/100
def calcLevel(df):
        global nivel_comb
        autonomia = random.uniform(1.6, 2.4)
        nivel_calc = (nivel_comb - ((df['distance']/1000)/autonomia))
        nivel_comb = nivel_calc
        return nivel_calc


df_apoio['nivel_litros'] = df_apoio.apply(lambda x: calcLevel(x), axis=1)
df_apoio['fuel_est'] = df_apoio.apply(lambda x: int(x['nivel_litros']*100/litragem_cam), axis=1)
display(df_apoio.head())
plotDataTimestamp(df_apoio)

Total Distance: 3281.38 m, Interval: 5.45 min,         Vel Media: 36.13 Km/h, Autonomia: 2.2 Km/L, max: 114, min: 5


,latitude,longitude,status,angle,fuel,weight,velocity,imei,date_created,distance,nivel_litros,fuel_est
6622,-3.879148,-38.761066,Parado,2.375795,96.0,-1.0,0.747417,25,2021-08-07 08:50:23,0.000000,201.600000,96
6623,-3.879255,-38.761116,Movimento,-7.255507,95.0,-1.0,16.074614,25,2021-08-07 08:50:28,13.127406,201.592838,95
6624,-3.880620,-38.761467,Movimento,6.199142,92.0,-1.0,47.606140,25,2021-08-07 08:50:54,156.696793,201.503250,95
6625,-3.883064,-38.761730,Parado,6.118005,92.0,-1.0,0.444744,25,2021-08-07 08:51:25,273.322582,201.337315,95
6626,-3.883070,-38.761730,Parado,6.175217,93.0,-1.0,0.245021,25,2021-08-07 08:51:31,0.667171,201.336997,95


### Trajeto ao Ponto de Carregamento

Método de Geração: Através dos dados reais de geolocalização é estimado o consumo de combustível do caminhão em Km,L.

In [ ]:
from geopy.distance import great_circle
from IPython.display import display, HTML
import random

class circleWay():
        def __init__(self, df):
                circleWay.generate(df)
        
        @classmethod
        def generate(cls, df):
                litragem_cam = 210
                df_apoio = df.copy()
                df_apoio = df_apoio.loc[(df['imei'] == caminhao) & 
                                        (df['date_created'] >= '2021-08-06 16:16:39') &
                                        (df['date_created'] <= '2021-08-06 16:32:09')]


                timestamp = df_apoio['date_created'].to_numpy()
                interval = float((timestamp[-1] - timestamp[0]) / (10**9))
                location = df_apoio.loc[:,['latitude','longitude']].to_numpy()
                distance = great_circle(location[0], location[-1]).m
                vel_media = (distance*3600)/(1000*interval)
                autonomia = (distance/1000)/(0.71*litragem_cam/100)
                print(f'Total Distance: {round(distance, 2)} m, Interval: {round((interval / 60), 2)} min, \
                        Vel Media: {round(vel_media, 2)} Km/h, Autonomia: {round(autonomia, 2)} Km/L, max: {int(df["fuel"].max())}, min: {int(df["fuel"].min())}')

                cls.last_lat = df_apoio['latitude'].to_numpy()[0]
                cls.last_lon = df_apoio['longitude'].to_numpy()[0]

                df_apoio['distance'] = df_apoio.apply(lambda x: circleWay.calcDistance(x), axis=1)

                cls.nivel_comb = df_apoio['fuel'].to_numpy()[0]*litragem_cam/100

                df_apoio['nivel_litros'] = df_apoio.apply(lambda x: calcLevel(x), axis=1)
                df_apoio['fuel_est'] = df_apoio.apply(lambda x: int(x['nivel_litros']*100/litragem_cam), axis=1)
                print(f'Distance Calc: {df_apoio["distance"].sum()}')
                return df_apoio
        
        @classmethod
        def calcDistance(cls, df):
                dist = great_circle([df['latitude'],df['longitude']], [cls.last_lat, cls.last_lon]).m
                cls.last_lat = df['latitude']
                cls.last_lon = df['longitude']
                return dist

        @classmethod
        def calcLevel(cls, df):
                autonomia = random.uniform(1.6, 2.4)
                nivel_calc = (cls.nivel_comb - ((df['distance']/1000)/autonomia))
                cls.nivel_comb = nivel_calc
                return nivel_calc

df_way = circleWay.generate(df)
display(df_way.head())
plotDataTimestamp(df_way)

Total Distance: 550.43 m, Interval: 15.5 min,                         Vel Media: 2.13 Km/h, Autonomia: 0.37 Km/L, max: 114, min: 5
Distance Calc: 817.2126977863818


,latitude,longitude,status,angle,fuel,weight,velocity,imei,date_created,distance,nivel_litros,fuel_est
3868,-3.909287,-38.758678,Movimento,6.057806,80.0,-1.0,16.226978,25,2021-08-06 16:16:39,0.000000,166.774652,79
3870,-3.909756,-38.759205,Movimento,-3.758087,81.0,-1.0,29.515764,25,2021-08-06 16:16:49,78.343146,166.735454,79
3872,-3.909950,-38.759506,Movimento,-10.048460,82.0,-1.0,30.370251,25,2021-08-06 16:16:54,39.753721,166.715680,79
3874,-3.910157,-38.759769,Movimento,3.384842,83.0,-1.0,26.639341,25,2021-08-06 16:17:00,37.162516,166.696213,79
3876,-3.910119,-38.760246,Movimento,1.779030,81.0,-1.0,27.506182,25,2021-08-06 16:17:10,53.085022,166.672555,79


### Calcula nova posição a partir da posição atual e um determinado deslocamento angular:
 - Parâmetros:
    - location: lista contendo a geolocalização 
    - dist: distancia em metros em relação ao ponto
 - Retorno:
    - Lista contendo a nova geolocalização decimal

In [ ]:
from geopy.distance import distance, great_circle
import random 

start_point = (-3.910157, -38.759769)
'''
    Retorna novo ponto com distancia 'dist' em metros em relação ao primeiro ponto 'location'
'''
def genNewLocation(location, dist):
    degress = [0,90,180,270]
    idx = random.randint(0,3)
    new_point = distance(meters=dist).destination(location, bearing=degress[idx])
    new_point_d = new_point.format_decimal() 
    new_point_f = [round(float(x), 6) for x in list(new_point_d.split(','))]
    return new_point_f
    
new_point = genNewLocation(start_point, 15)    
print(new_point)
print(great_circle((-3.910157, -38.759769), new_point).m)

[-3.910157, -38.759904]
14.976393003006738


### Geração de Dados de Carregamento

Método de geração: Estima-se a quantidade de carregamentos realizados pela escavadeira com base no volume da caçamba do caminhão e na capacidade da concha da escavadeira. Os dados de combustível são gerados a partir da autonomia, em L/h, da escavadeira. O tempo de carregamento é calculado através do intervalo estimado de cada 'conchada'.    

Dados da Escavadeira:
 - Modelo: Cat320D
  - Bateria
  - Capacidade Combustível: 345 L
  - Capascidade da Caçamba: 732 Kg (1,19 m³)
  - Autonomia: 9 L/h a 12 L/h
  - Tempo de Carga de um Caminhao: 4min a 8min



In [ ]:
import random
from datetime import datetime, timedelta

start_point = (-3.908756, -38.763611)
start_time = '2021-08-07 14:00:00'
start_time_date = datetime.fromisoformat(start_time)
print(start_time_date)
litragem_esc = 345
start_fuel = fuel = 345
weight_sum = 0
weights = []
lats = []
longs = []
dates =[]
intervals = [] #intervalo em segundos entre as conchadas
fuels = [] #combustivel em litros da escavadeira
total_weight = random.randint(9470, 14350) # peso estimado suportado pela caçamba do caminhão.


while weight_sum < total_weight:
    w = random.randint(480,640) # peso estimado coletado pela concha da escavadeira.
    weight_sum += w
    weights.append(w)
    interval = random.randint(12,19)
    intervals.append(interval) # tempo considerado para coletar uma amostra e depositá-la no caminhão.
    # Calculo baseado na autonomia da escavadeira que varia entre 9 a 12 L/h
    fuel -= random.randint(9, 12)*interval/3600 
    fuels.append(fuel)

for idx in range(len(weights)):
    # distancia em metros em relacao ao ponto geografico inicial. 
    # Isto simula o erro inerente ao gps, considerando o veículo no estado parado. 
    _dist = random.uniform(0.4,1.3) 
    new_point = genNewLocation(start_point, _dist)
    (lat, lon) = tuple(new_point)
    lats.append(lat)
    longs.append(lon)


df_dict = {
    'latitude': lats,
    'longitude': longs,
    'weight': weights,
    'interval': intervals,
    'fuel_levels':fuels
}   

last_time = start_time_date
def updateTime(df):
    global last_time
    curr_time = last_time + timedelta(seconds = df['interval'])
    last_time = curr_time
    return curr_time

velocities = [round(random.randint(0,0), 2) for _ in range(len(weights))]
df_esc = pd.DataFrame(df_dict)
# df_esc['fuel'] = df_esc['fuel_levels']*100/litragem_esc
df_esc['fuel'] = df_esc['fuel_levels'].apply(lambda x: int(x*100/litragem_esc))
df_esc['date_created'] = df_esc.apply(lambda x: updateTime(x), axis=1)
df_esc['velocity'] = pd.Series(velocities)
df_esc['status'] = 'Carregando'
print(f'Peso total: {df_esc["weight"].sum()} Kg, Tempo total: {round(df_esc["interval"].sum()/60, 2)} min, Total amostras: {df_esc["weight"].count()}')
display(df_esc)
plotDataTimestamp(df_esc)

2021-08-07 14:00:00
Peso total: 10328 Kg, Tempo total: 4.83 min, Total amostras: 19


,latitude,longitude,weight,interval,fuel_levels,fuel,date_created,velocity,status
0,-3.908756,-38.763602,560,14,344.957222,99,2021-08-07 14:00:14,0,Carregando
1,-3.908744,-38.763611,607,14,344.918333,99,2021-08-07 14:00:28,0,Carregando
2,-3.908766,-38.763611,516,12,344.888333,99,2021-08-07 14:00:40,0,Carregando
3,-3.908756,-38.763601,549,15,344.842500,99,2021-08-07 14:00:55,0,Carregando
4,-3.908756,-38.763618,503,14,344.803611,99,2021-08-07 14:01:09,0,Carregando
5,-3.908756,-38.763621,638,12,344.773611,99,2021-08-07 14:01:21,0,Carregando
6,-3.908751,-38.763611,522,15,344.723611,99,2021-08-07 14:01:36,0,Carregando
7,-3.908766,-38.763611,591,18,344.678611,99,2021-08-07 14:01:54,0,Carregando
8,-3.908756,-38.763600,509,19,344.620556,99,2021-08-07 14:02:13,0,Carregando
9,-3.908761,-38.763611,501,14,344.585556,99,2021-08-07 14:02:27,0,Carregando


In [ ]:
import random
import numpy as np
from datetime import datetime, timedelta
from geopy.distance import distance, great_circle

class circleLoading():
    def __init__(self, df_main):
        circleLoading.generate(df_main)

    @classmethod
    def generate(cls, df_main):
        cls.df_main = df_main
        last_row = df_main.tail(1)
        start_point = (last_row['latitude'].values[0], last_row['longitude'].values[0])
        # start_time = last_row['date_created'].values
        # start_time_date = pd.Timestamp(last_row['date_created'].values[0]).to_datetime64()
        start_time_date = last_row['date_created'].values[0]
        cls.last_time = pd.to_datetime(start_time_date)
        # print(last_time)
        litragem_esc = 345
        fuel = 345
        weight_sum = 0
        weights = []
        lats = []
        longs = []
        intervals = [] #intervalo em segundos entre as conchadas
        fuels = [] #combustivel em litros da escavadeira
        total_weight = random.randint(9470, 14350) # peso estimado suportado pela caçamba do caminhão.


        while weight_sum < total_weight:
            w = random.randint(480,640) # peso estimado coletado pela concha da escavadeira.
            weight_sum += w
            weights.append(w)
            interval = random.randint(12,19)
            intervals.append(interval) # tempo considerado para coletar uma amostra e depositá-la no caminhão.
            # Calculo baseado na autonomia da escavadeira que varia entre 9 a 12 L/h
            fuel -= random.randint(9, 12)*interval/3600 
            fuels.append(fuel)

        for _ in range(len(weights)):
            # distancia em metros em relacao ao ponto geografico inicial. 
            # Isto simula o erro inerente ao gps, considerando o veículo no estado parado. 
            _dist = random.uniform(0.4,1.3) 
            new_point = genNewLocation(start_point, _dist)
            (lat, lon) = tuple(new_point)
            lats.append(lat)
            longs.append(lon)


        df_dict = {
            'latitude': lats,
            'longitude': longs,
            'weight': weights,
            'interval': intervals,
            'fuel_levels':fuels
        }   

        velocities = [round(random.randint(0,0), 2) for _ in range(len(weights))]
        df_new = pd.DataFrame(df_dict)
        # df_esc['fuel'] = df_esc['fuel_levels']*100/litragem_esc
        df_new['fuel'] = df_new['fuel_levels'].apply(lambda x: int(x*100/litragem_esc))
        df_new['date_created'] = df_new.apply(lambda x: circleLoading.updateTime(x), axis=1)
        df_new['velocity'] = pd.Series(velocities)
        df_new['status'] = 'Carregando'
        print(f'Peso total: {df_new["weight"].sum()} Kg, Tempo total: {round(df_new["interval"].sum()/60, 2)} min, Total amostras: {df_new["weight"].count()}')
        # display(df_esc)
        # plotDataTimestamp(df_esc)
        return df_new
    
    @classmethod
    def updateTime(cls, df):
        # print(f'las_time: {cls.last_time}, type: {type(cls.last_time)}')
        curr_time = cls.last_time + np.timedelta64(int(df['interval']),'s')
        cls.last_time = curr_time
        return curr_time

    @classmethod
    def addOffsetLocation(cls, df):
        degress = [0,90,180,270]
        idx = random.randint(0,3)
        location = (df['latitude'],df['longitude'])
        new_point = distance(meters=3).destination(location, bearing=degress[idx])
        new_point_d = new_point.format_decimal() 
        new_point_f = [round(float(x), 6) for x in list(new_point_d.split(','))]
        return new_point_f

    @classmethod
    def convertToTruck(cls, df, fuel_init):
        res = df.apply(lambda x: circleLoading.addOffsetLocation(x), axis=1)
        df['latitude'] = res.apply(lambda x: x[0])
        df['longitude'] = res.apply(lambda x: x[0])
        df['weight'] = -1
        fuel_truck = []
        for _ in range(df['date_created'].count()):
            fuel_init -= random.randint(9, 12)*interval/3600 
            fuel_truck.append(fuel_init)
        df['fuel_levels'] = pd.Series(fuel_truck)
        # df['fuel'] = df['fuel_levels'].apply(lambda x: int(x*100/litragem_cam))
        display(df)


display(df_apoio.tail(1))
df_return = circleLoading.generate(df_apoio)
df_apoio = df_apoio.append(df_return, ignore_index=True)
display(df_return)
circleLoading.convertToTruck(df_return, fuel_init = 200)

,latitude,longitude,status,angle,fuel,weight,velocity,imei,date_created,distance,nivel_litros,fuel_est,interval,fuel_levels
1495,-3.908813,-38.76366,Carregando,NaN,99.0,629.0,0.0,NaN,2021-08-06 22:22:07,NaN,NaN,NaN,19.0,344.068889


Peso total: 12068 Kg, Tempo total: 5.58 min, Total amostras: 22


,latitude,longitude,weight,interval,fuel_levels,fuel,date_created,velocity,status
0,-3.908823,-38.763660,481,18,344.955000,99,2021-08-06 22:22:25,0,Carregando
1,-3.908813,-38.763665,501,15,344.909167,99,2021-08-06 22:22:40,0,Carregando
2,-3.908824,-38.763660,513,15,344.859167,99,2021-08-06 22:22:55,0,Carregando
3,-3.908804,-38.763660,617,14,344.816389,99,2021-08-06 22:23:09,0,Carregando
4,-3.908813,-38.763656,486,17,344.769167,99,2021-08-06 22:23:26,0,Carregando
5,-3.908813,-38.763669,527,18,344.709167,99,2021-08-06 22:23:44,0,Carregando
6,-3.908803,-38.763660,497,16,344.655833,99,2021-08-06 22:24:00,0,Carregando
7,-3.908802,-38.763660,580,17,344.608611,99,2021-08-06 22:24:17,0,Carregando
8,-3.908824,-38.763660,628,17,344.551944,99,2021-08-06 22:24:34,0,Carregando
9,-3.908821,-38.763660,600,13,344.515833,99,2021-08-06 22:24:47,0,Carregando


,latitude,longitude,weight,interval,fuel_levels,fuel,date_created,velocity,status
0,-3.908823,-3.908823,-1,18,197.158333,99,2021-08-06 22:22:25,0,Carregando
1,-3.908813,-3.908813,-1,15,194.316667,99,2021-08-06 22:22:40,0,Carregando
2,-3.908824,-3.908824,-1,15,191.733333,99,2021-08-06 22:22:55,0,Carregando
3,-3.908804,-3.908804,-1,14,188.633333,99,2021-08-06 22:23:09,0,Carregando
4,-3.908813,-3.908813,-1,17,185.791667,99,2021-08-06 22:23:26,0,Carregando
5,-3.908840,-3.908840,-1,18,182.950000,99,2021-08-06 22:23:44,0,Carregando
6,-3.908776,-3.908776,-1,16,179.850000,99,2021-08-06 22:24:00,0,Carregando
7,-3.908802,-3.908802,-1,17,177.266667,99,2021-08-06 22:24:17,0,Carregando
8,-3.908824,-3.908824,-1,17,174.166667,99,2021-08-06 22:24:34,0,Carregando
9,-3.908821,-3.908821,-1,13,171.066667,99,2021-08-06 22:24:47,0,Carregando


In [ ]:
class genPath():
    def __init__(self) -> None:
        pass

    @classmethod
    def gen(cls, df, loaded=False):
        # loaded: se o caminhao estiver carregado adiciona fator multiplicador no consumo
        velocity = round(random.uniform(3,9),1) #em Km/h
        distance = round(random.uniform(0.8,1.6),2) #em Km
        consumption = round(random.uniform(1.5, 3),1)*1.15 if loaded == True else round(random.uniform(1.5, 3),1) #em Km/L do caminhao
        interval = round((distance/velocity)*60,2) #em min
        
        status = "Movimento"
        date_created = df['date_created'].tail(1).values[0] + np.timedelta64(int(interval),'m')

        # Variáveis Caminhao 
        fuel_consumed = round(distance/consumption,2)
        fuel_level = df['fuel_level'].tail(1).values[0] - fuel_consumed
        fuel = int(fuel_level*100/litragem_cam)

        # Variáveis Escavadeira 
        # Com base na autonomia média da escavadeira. Fator multiplicador pois ela se encontra parada enquanto o caminhao se locomove
        fuel_consumed_esc = (random.randint(9, 12)*interval/60)*0.91 
        fuel_level_esc = df['fuel_level_esc'].tail(1).values[0] - fuel_consumed_esc
        fuel_esc = int(fuel_level_esc*100/litragem_esc)
        weight = 0

        df_dict = {
            'date_created': [date_created],
            'status': [status],
            'fuel': [fuel],
            'weight': [weight],
            'distance': [distance], 
            'fuel_level': [fuel_level],
            'fuel_comsumed': [fuel_consumed], 
            'interval': [interval], 
            'fuel_esc': [fuel_esc],
            'fuel_level_esc': [fuel_level_esc],
            'fuel_comsumed_esc': [fuel_consumed_esc]
        }

        df_res = pd.DataFrame(df_dict)
        return df_res

In [ ]:
class genLoading():
    def __init__(self) -> None:
        pass

    @classmethod
    def gen(cls, df):
        auton_cam = round(random.uniform(0.3, 5.2),1) #em L/h do caminhao no estado parado
        capacity_weight = random.randint(9470, 14350) # peso estimado suportado pela caçamba do caminhão
        last_date = df['date_created'].tail(1).values[0]
        last_fuel = df['fuel_level'].tail(1).values[0]
        last_fuel_esc = df['fuel_level_esc'].tail(1).values[0]
        payload_sum = 0

        # Arrays de dados
        status = []
        dates_created = []
        fuels = []
        weights = []
        fuel_levels = []
        fuel_comsumeds = []
        intervals = []
        fuel_escs = []
        fuel_level_escs = []
        fuel_comsumed_escs = []
        distance = []

        while payload_sum < capacity_weight:
            payload_esc = random.randint(480,640) # peso estimado coletado pela concha da escavadeira
            weights.append(payload_esc)
            payload_sum += payload_esc
            interval = random.randint(9,19) # tempo em segundos (s) considerado para coletar uma amostra e depositá-la no caminhão.
            intervals.append(interval/60)
            date_created = last_date + np.timedelta64(interval,'s')
            dates_created.append(date_created)
            last_date = date_created
            status.append("Carregando")
            distance.append(0)
            
            # Variáveis Caminhao 
            fuel_consumed = (auton_cam*interval)/3600
            fuel_level = last_fuel - fuel_consumed
            last_fuel = fuel_level
            fuel = int(fuel_level*100/litragem_cam)
            fuel_comsumeds.append(fuel_consumed)
            fuel_levels.append(fuel_level)
            fuels.append(fuel)


            # Variáveis Escavadeira 
            # Consumo calculado com base na autonomia média da escavadeira.
            fuel_consumed_esc = (random.randint(9, 12)*interval/3600)
            fuel_level_esc = last_fuel_esc - fuel_consumed_esc
            last_fuel_esc = fuel_level_esc
            fuel_esc = int(fuel_level_esc*100/litragem_esc)
            fuel_escs.append(fuel_esc)
            fuel_level_escs.append(last_fuel_esc)
            fuel_comsumed_escs.append(fuel_consumed_esc)


        df_dict = {
            'date_created': dates_created,
            'status': status,
            'fuel': fuels,
            'weight': weights,
            'distance': distance, 
            'fuel_level': fuel_levels,
            'fuel_comsumed': fuel_comsumeds, 
            'interval': intervals, 
            'fuel_esc': fuel_escs,
            'fuel_level_esc': fuel_level_escs,
            'fuel_comsumed_esc': fuel_comsumed_escs
        }

        df_res = pd.DataFrame(df_dict)
        return df_res

In [ ]:
class genDischarging():
    def __init__(self) -> None:
        pass

    @classmethod
    def gen(cls, df):
        auton_cam = round(random.uniform(0.3, 5.2),1)*1.05 #em L/h do caminhao no estado parado, fator multiplicador considerando carga na caçamba
        distance = 0 #em Km
        interval = random.randint(3,6) #em min
        
        status = "Descarregando"
        date_created = df['date_created'].tail(1).values[0] + np.timedelta64(int(interval),'m')

        # Variáveis Caminhao 
        fuel_consumed = (auton_cam*interval)/60
        fuel_level = df['fuel_level'].tail(1).values[0] - fuel_consumed
        fuel = int(fuel_level*100/litragem_cam)

        # Variáveis Escavadeira 
        # Com base na autonomia média da escavadeira. Fator multiplicador pois ela se encontra parada enquanto o caminhao se locomove, 
        # logo seu consumo é menor em relação ao estado em que se encontra operando.
        fuel_consumed_esc = (random.randint(9, 12)*interval/60)*0.91 
        fuel_level_esc = df['fuel_level_esc'].tail(1).values[0] - fuel_consumed_esc
        fuel_esc = int(fuel_level_esc*100/litragem_esc)
        weight = 0

        df_dict = {
            'date_created': [date_created],
            'status': [status],
            'fuel': [fuel],
            'weight': [weight],
            'distance': [distance], 
            'fuel_level': [fuel_level],
            'fuel_comsumed': [fuel_consumed], 
            'interval': [interval], 
            'fuel_esc': [fuel_esc],
            'fuel_level_esc': [fuel_level_esc],
            'fuel_comsumed_esc': [fuel_consumed_esc]
        }

        df_res = pd.DataFrame(df_dict)
        return df_res

In [ ]:
def genOneCycle(df_cycle):
    # Ciclo de Uma Operação
    df_cycle = df_cycle.append(genPath.gen(df_cycle), ignore_index=True) #Caminho até a escavadeira
    df_cycle = df_cycle.append(genLoading.gen(df_cycle), ignore_index=True) #Ciclo de Carregamento
    df_cycle = df_cycle.append(genPath.gen(df_cycle, loaded=True), ignore_index=True) #Caminho até o ponto de Descarregamento
    df_cycle = df_cycle.append(genDischarging.gen(df_cycle), ignore_index=True) #Ciclo de Descarregamento
    return df_cycle


################## Inicio #########################################
day_list = []
date = pd.to_datetime(f'2021-08-06 07:00:00') 
qtde_dias = 18 #Quantidade de dias úteis gerados
for idx in range(qtde_dias):
    minutes = int(random.uniform(-1, 1)*57)
    date += np.timedelta64(int(minutes*60),'s')
    day_list.append(str(date))
    date += np.timedelta64(1,'D') if date.weekday() < 4 else np.timedelta64(3,'D')


df_dict = {
            'date_created': [pd.to_datetime(day_list[0])],
            'status': ['Parado'],
            'fuel': [98],
            'weight': [0],
            'distance': [0], 
            'fuel_level': [190],
            'fuel_comsumed': [0], 
            'interval': [0],
            'fuel_esc': [95],
            'fuel_level_esc': [327],
            'fuel_comsumed_esc': [0]
        }


df_cycle = pd.DataFrame(df_dict)

#Ciclo de 'day_list' dias
for date in day_list:
    fuel_init = random.randint(86,97)
    fuel_level_init = (fuel_init*litragem_cam)/100
    fuel_esc_init = random.randint(88,95)
    fuel_level_esc_init = (fuel_esc_init*litragem_esc)/100

    day_init = {
        'date_created': pd.to_datetime(date),
        'status': 'Parado',
        'fuel': fuel_init,
        'weight': 0,
        'distance': 0, 
        'fuel_level': fuel_level_init,
        'fuel_comsumed': 0, 
        'interval': 0,
        'fuel_esc': fuel_esc_init,
        'fuel_level_esc': fuel_level_esc_init,
        'fuel_comsumed_esc': 0
    }

    df_cycle = df_cycle.append(day_init, ignore_index=True)
    print(f'Inicio do Turno: {df_cycle.tail(1)}')

    #Ciclo de um Dia
    for turno in range(2):
        #Ciclo de um Turno
        for _ in range(7):
            df_cycle = genOneCycle(df_cycle)

        display(df_cycle)
        print(f'Turno {turno+1}: time interval: {df_cycle.iloc[-1,0] - df_cycle.iloc[0,0]}, interval: {round(df_cycle["interval"].sum()/60,2)} h, total_weight: {df_cycle["weight"].sum()} Kg')
        
        #Intervalo de Almoço
        df_dict_interval = {
                'date_created': [df_cycle['date_created'].tail(1).values[0] + np.timedelta64(int(random.randint(57,83)),'m')],
                'status': ['Parado'],
                'fuel': [df_cycle['fuel'].tail(1).values[0]],
                'weight': [0],
                'distance': [0], 
                'fuel_level': [df_cycle['fuel_level'].tail(1).values[0]],
                'fuel_comsumed': [0], 
                'interval': [0],
                'fuel_esc': [df_cycle['fuel_esc'].tail(1).values[0]],
                'fuel_level_esc': [df_cycle['fuel_level_esc'].tail(1).values[0]],
                'fuel_comsumed_esc': [0]
            }
        df_interval = pd.DataFrame(df_dict_interval)
        df_cycle = df_cycle.append(df_interval, ignore_index=True)

df_cycle.to_csv(f"..{sep}dados_csv{sep}dados_sim_{random.randint(0,1000)}.csv")

Inicio do Turno:          date_created  status  fuel  weight  distance  fuel_level  \
1 2021-08-06 07:53:00  Parado    91       0         0       191.1   

   fuel_comsumed  interval  fuel_esc  fuel_level_esc  fuel_comsumed_esc  
1              0         0        93          320.85                  0  


,date_created,status,fuel,weight,distance,fuel_level,fuel_comsumed,interval,fuel_esc,fuel_level_esc,fuel_comsumed_esc
0,2021-08-06 07:53:00,Parado,98,0,0.00,190.000000,0.000000,0.000000,95,327.000000,0.000000
1,2021-08-06 07:53:00,Parado,91,0,0.00,191.100000,0.000000,0.000000,93,320.850000,0.000000
2,2021-08-06 08:00:00,Movimento,90,0,0.95,190.690000,0.410000,7.600000,92,319.697333,1.152667
3,2021-08-06 08:00:17,Carregando,90,562,0.00,190.666389,0.023611,0.283333,92,319.645389,0.051944
4,2021-08-06 08:00:26,Carregando,90,597,0.00,190.653889,0.012500,0.150000,92,319.617889,0.027500
...,...,...,...,...,...,...,...,...,...,...,...
179,2021-08-06 11:11:14,Carregando,86,559,0.00,181.866056,0.016333,0.233333,83,288.625884,0.038889
180,2021-08-06 11:11:26,Carregando,86,592,0.00,181.852056,0.014000,0.200000,83,288.595884,0.030000
181,2021-08-06 11:11:38,Carregando,86,581,0.00,181.838056,0.014000,0.200000,83,288.565884,0.030000
182,2021-08-06 11:20:38,Movimento,86,0,1.37,181.398056,0.440000,9.560000,83,287.115951,1.449933


Turno 1: time interval: 0 days 03:32:38, interval: 3.66 h, total_weight: 90306 Kg


,date_created,status,fuel,weight,distance,fuel_level,fuel_comsumed,interval,fuel_esc,fuel_level_esc,fuel_comsumed_esc
0,2021-08-06 07:53:00,Parado,98,0,0.00,190.000000,0.000000,0.000000,95,327.000000,0.000000
1,2021-08-06 07:53:00,Parado,91,0,0.00,191.100000,0.000000,0.000000,93,320.850000,0.000000
2,2021-08-06 08:00:00,Movimento,90,0,0.95,190.690000,0.410000,7.600000,92,319.697333,1.152667
3,2021-08-06 08:00:17,Carregando,90,562,0.00,190.666389,0.023611,0.283333,92,319.645389,0.051944
4,2021-08-06 08:00:26,Carregando,90,597,0.00,190.653889,0.012500,0.150000,92,319.617889,0.027500
...,...,...,...,...,...,...,...,...,...,...,...
351,2021-08-06 16:11:24,Carregando,81,550,0.00,171.089611,0.008000,0.150000,72,250.575529,0.022500
352,2021-08-06 16:11:35,Carregando,81,593,0.00,171.079833,0.009778,0.183333,72,250.544974,0.030556
353,2021-08-06 16:11:54,Carregando,81,511,0.00,171.062944,0.016889,0.316667,72,250.486918,0.058056
354,2021-08-06 16:20:54,Movimento,81,0,1.11,170.462944,0.600000,9.650000,72,248.876977,1.609942


Turno 2: time interval: 0 days 08:32:54, interval: 7.76 h, total_weight: 172933 Kg
Inicio do Turno:            date_created  status  fuel  weight  distance  fuel_level  \
357 2021-08-09 07:10:00  Parado    97       0       0.0       203.7   

     fuel_comsumed  interval  fuel_esc  fuel_level_esc  fuel_comsumed_esc  
357            0.0       0.0        89          307.05                0.0  


...

Turno 2: time interval: 24 days 11:26:11, interval: 143.38 h, total_weight: 2916543 Kg
Inicio do Turno:             date_created  status  fuel  weight  distance  fuel_level  \
5979 2021-08-31 09:59:00  Parado    88       0       0.0       184.8   

      fuel_comsumed  interval  fuel_esc  fuel_level_esc  fuel_comsumed_esc  
5979            0.0       0.0        91          313.95                0.0  


,date_created,status,fuel,weight,distance,fuel_level,fuel_comsumed,interval,fuel_esc,fuel_level_esc,fuel_comsumed_esc
0,2021-08-06 07:53:00,Parado,98,0,0.00,190.000000,0.000000,0.000000,95,327.000000,0.000000
1,2021-08-06 07:53:00,Parado,91,0,0.00,191.100000,0.000000,0.000000,93,320.850000,0.000000
2,2021-08-06 08:00:00,Movimento,90,0,0.95,190.690000,0.410000,7.600000,92,319.697333,1.152667
3,2021-08-06 08:00:17,Carregando,90,562,0.00,190.666389,0.023611,0.283333,92,319.645389,0.051944
4,2021-08-06 08:00:26,Carregando,90,597,0.00,190.653889,0.012500,0.150000,92,319.617889,0.027500
...,...,...,...,...,...,...,...,...,...,...,...
6149,2021-08-31 13:46:24,Carregando,83,525,0.00,174.860833,0.018222,0.266667,80,276.844569,0.053333
6150,2021-08-31 13:46:33,Carregando,83,529,0.00,174.850583,0.010250,0.150000,80,276.817069,0.027500
6151,2021-08-31 13:46:47,Carregando,83,582,0.00,174.834639,0.015944,0.233333,80,276.770402,0.046667
6152,2021-08-31 13:56:47,Movimento,82,0,1.51,174.264639,0.570000,10.300000,79,274.895802,1.874600


Turno 1: time interval: 25 days 06:08:47, interval: 147.52 h, total_weight: 3002511 Kg


,date_created,status,fuel,weight,distance,fuel_level,fuel_comsumed,interval,fuel_esc,fuel_level_esc,fuel_comsumed_esc
0,2021-08-06 07:53:00,Parado,98,0,0.00,190.000000,0.000000,0.000000,95,327.000000,0.000000
1,2021-08-06 07:53:00,Parado,91,0,0.00,191.100000,0.000000,0.000000,93,320.850000,0.000000
2,2021-08-06 08:00:00,Movimento,90,0,0.95,190.690000,0.410000,7.600000,92,319.697333,1.152667
3,2021-08-06 08:00:17,Carregando,90,562,0.00,190.666389,0.023611,0.283333,92,319.645389,0.051944
4,2021-08-06 08:00:26,Carregando,90,597,0.00,190.653889,0.012500,0.150000,92,319.617889,0.027500
...,...,...,...,...,...,...,...,...,...,...,...
6319,2021-08-31 18:52:02,Carregando,78,591,0.00,165.417750,0.001750,0.150000,69,239.416983,0.022500
6320,2021-08-31 18:52:18,Carregando,78,569,0.00,165.414639,0.003111,0.266667,69,239.368094,0.048889
6321,2021-08-31 18:52:37,Carregando,78,583,0.00,165.410944,0.003694,0.316667,69,239.304761,0.063333
6322,2021-08-31 19:08:37,Movimento,78,0,1.32,164.950944,0.460000,16.850000,68,237.004736,2.300025


Turno 2: time interval: 25 days 11:19:37, interval: 151.45 h, total_weight: 3086061 Kg
